In [1]:
from PIL import Image
import cv2
import numpy as np
import matplotlib.pyplot as plt

- 흑백으로 변환
- 색상 반전
- crop (1080*1080)
- resize (1080\*1080) ==> (16*16)

### 이미지 path 저장

In [2]:
import os
dirs = os.getcwd() + '/img/'
img_original = os.listdir(dirs)
img_original = [file for file in img_original if 'png' in file]
img_original

['20201125_174207.mp4_000000344.png',
 '20201125_174207.mp4_000000444.png',
 '20201125_174207.mp4_000000577.png',
 '20201125_174207.mp4_000000677.png',
 '20201125_174207.mp4_000000777.png',
 '20201125_174207.mp4_000000877.png',
 '20201125_174207.mp4_000000977.png',
 '20201125_174207.mp4_000001077.png',
 '20201125_174207.mp4_000001177.png',
 '20201125_174207.mp4_000001277.png',
 '20201125_174207.mp4_000001410.png',
 '20201125_174207.mp4_000001510.png',
 '20201125_174207.mp4_000001610.png',
 '20201125_174207.mp4_000001710.png',
 '20201125_174207.mp4_000001810.png',
 '20201125_174207.mp4_000001910.png',
 '20201125_174207.mp4_000002010.png',
 '20201125_174207.mp4_000002110.png',
 '20201125_174207.mp4_000002243.png',
 '20201125_174207.mp4_000002343.png',
 '20201125_174207.mp4_000002443.png',
 '20201125_174207.mp4_000002543.png',
 '20201125_174207.mp4_000002643.png',
 '20201125_174207.mp4_000002743.png',
 '20201125_174207.mp4_000002843.png',
 '20201125_174207.mp4_000002943.png',
 '20201125_1

In [3]:
len(img_original)

1131

### 이미지 변환

In [4]:
dirs_dest = os.getcwd() + '/img_converted/'
img_converted = []
for img_name in img_original:
    img = cv2.imread(dirs+img_name, cv2.IMREAD_GRAYSCALE)
    # 아랫부분을 중심으로 1080*1080으로 자르기
    crop_img = img[840:, :]
    # 16*16으로 줄이기
    resize_img = cv2.resize(crop_img, (16,16), interpolation=cv2.INTER_AREA)
    # 색상 반전
    for x in range(16):
        for y in range(16):
            if resize_img[y, x] < 128:
                resize_img[y, x] = 255
            else:
                resize_img[y, x] = 0
    # local에 저장
    cv2.imwrite(dirs_dest+img_name, resize_img)
    # list에 저장
    img_converted.append(resize_img)

In [14]:
'''
img = cv2.imread(dirs+img_original[0], cv2.IMREAD_GRAYSCALE)
crop_img = img[840:, :]
cv2.imwrite('cropped.png', crop_img)
resize_img = cv2.resize(crop_img, (16,16), interpolation=cv2.INTER_AREA)
cv2.imwrite('resized.png', resize_img)
reverse_img = cv2.bitwise_not(resize_img)
#cv2.imwrite('reversed.png', reverse_img)
for x in range(16):
    for y in range(16):
        if reverse_img[y, x] < 128:
            reverse_img[y, x] = 0
        else:
            reverse_img[y, x] = 255
cv2.imwrite('reversed_contrast.png', reverse_img)
'''
#print(np.array(reverse_img,dtype=np.uint8))

True

In [5]:
len(img_converted)

1131

### xml 불러오기 & data_additional list 만들기
- [(방향, 속도, [img]), ..., (방향, 속도, [img])]

In [6]:
dirs_label = os.getcwd() + '/label/'
label = os.listdir(dirs_label)
label = [file for file in label if 'xml' in file]
label

['20201125_174207.mp4_000000344.xml',
 '20201125_174207.mp4_000000444.xml',
 '20201125_174207.mp4_000000577.xml',
 '20201125_174207.mp4_000000677.xml',
 '20201125_174207.mp4_000000777.xml',
 '20201125_174207.mp4_000000877.xml',
 '20201125_174207.mp4_000000977.xml',
 '20201125_174207.mp4_000001077.xml',
 '20201125_174207.mp4_000001177.xml',
 '20201125_174207.mp4_000001277.xml',
 '20201125_174207.mp4_000001410.xml',
 '20201125_174207.mp4_000001510.xml',
 '20201125_174207.mp4_000001610.xml',
 '20201125_174207.mp4_000001710.xml',
 '20201125_174207.mp4_000001810.xml',
 '20201125_174207.mp4_000001910.xml',
 '20201125_174207.mp4_000002010.xml',
 '20201125_174207.mp4_000002110.xml',
 '20201125_174207.mp4_000002243.xml',
 '20201125_174207.mp4_000002343.xml',
 '20201125_174207.mp4_000002443.xml',
 '20201125_174207.mp4_000002543.xml',
 '20201125_174207.mp4_000002643.xml',
 '20201125_174207.mp4_000002743.xml',
 '20201125_174207.mp4_000002843.xml',
 '20201125_174207.mp4_000002943.xml',
 '20201125_1

In [7]:
len(label)

1131

In [8]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

In [10]:
data_additional = []
failed = []
#for file, img in zip(label, img_converted):
for i in range(len(label)):
    file = label[i]
    img = img_converted[i]
    # label(direction)
    doc = ET.parse(dirs_label+file)
    root = doc.getroot()
    object = root.findall("object")
    try:
        name = [x.findtext("name") for x in object]
        direction = float(name[0])
    except:
        print("["+str(i)+"/"+str(len(label))+"] 실패")
        failed.append(label[i])
        continue
    # velocity는 0
    # img는 img
    
    # list에 저장
    data_additional.append((direction, 0, img))
    print("["+str(i+1)+"/"+str(len(label))+"] 완료")

[1/1131] 완료
[2/1131] 완료
[3/1131] 완료
[4/1131] 완료
[5/1131] 완료
[6/1131] 완료
[7/1131] 완료
[8/1131] 완료
[9/1131] 완료
[10/1131] 완료
[11/1131] 완료
[12/1131] 완료
[13/1131] 완료
[14/1131] 완료
[15/1131] 완료
[16/1131] 완료
[17/1131] 완료
[18/1131] 완료
[19/1131] 완료
[20/1131] 완료
[21/1131] 완료
[22/1131] 완료
[23/1131] 완료
[24/1131] 완료
[25/1131] 완료
[26/1131] 완료
[27/1131] 완료
[28/1131] 완료
[29/1131] 완료
[30/1131] 완료
[31/1131] 완료
[32/1131] 완료
[33/1131] 완료
[34/1131] 완료
[35/1131] 완료
[36/1131] 완료
[37/1131] 완료
[38/1131] 완료
[39/1131] 완료
[40/1131] 완료
[41/1131] 완료
[42/1131] 완료
[43/1131] 완료
[44/1131] 완료
[45/1131] 완료
[46/1131] 완료
[47/1131] 완료
[48/1131] 완료
[49/1131] 완료
[50/1131] 완료
[51/1131] 완료
[52/1131] 완료
[53/1131] 완료
[54/1131] 완료
[55/1131] 완료
[56/1131] 완료
[57/1131] 완료
[58/1131] 완료
[59/1131] 완료
[60/1131] 완료
[61/1131] 완료
[62/1131] 완료
[63/1131] 완료
[64/1131] 완료
[65/1131] 완료
[66/1131] 완료
[67/1131] 완료
[68/1131] 완료
[69/1131] 완료
[70/1131] 완료
[71/1131] 완료
[72/1131] 완료
[73/1131] 완료
[74/1131] 완료
[75/1131] 완료
[76/1131] 완료
[77/1131] 완료
[78/1131

[668/1131] 완료
[669/1131] 완료
[670/1131] 완료
[671/1131] 완료
[672/1131] 완료
[673/1131] 완료
[674/1131] 완료
[675/1131] 완료
[676/1131] 완료
[677/1131] 완료
[678/1131] 완료
[679/1131] 완료
[680/1131] 완료
[681/1131] 완료
[682/1131] 완료
[683/1131] 완료
[684/1131] 완료
[685/1131] 완료
[686/1131] 완료
[687/1131] 완료
[688/1131] 완료
[689/1131] 완료
[690/1131] 완료
[691/1131] 완료
[692/1131] 완료
[693/1131] 완료
[694/1131] 완료
[695/1131] 완료
[696/1131] 완료
[697/1131] 완료
[698/1131] 완료
[699/1131] 완료
[700/1131] 완료
[701/1131] 완료
[702/1131] 완료
[703/1131] 완료
[704/1131] 완료
[705/1131] 완료
[706/1131] 완료
[707/1131] 완료
[708/1131] 완료
[709/1131] 완료
[710/1131] 완료
[711/1131] 완료
[712/1131] 완료
[713/1131] 완료
[714/1131] 완료
[715/1131] 완료
[716/1131] 완료
[717/1131] 완료
[718/1131] 완료
[719/1131] 완료
[720/1131] 완료
[721/1131] 완료
[722/1131] 완료
[723/1131] 완료
[724/1131] 완료
[725/1131] 완료
[726/1131] 완료
[727/1131] 완료
[728/1131] 완료
[729/1131] 완료
[730/1131] 완료
[731/1131] 완료
[732/1131] 완료
[733/1131] 완료
[734/1131] 완료
[735/1131] 완료
[736/1131] 완료
[737/1131] 완료
[738/1131] 완료
[739/1

In [11]:
len(data_additional)

1131

In [12]:
failed

[]

In [13]:
data_additional

[(0.0,
  0,
  array([[  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255,   0,   0,   0,
            0,   0,   0],
         [  0,   0,   0,   0,   0,   0,   0, 255, 255, 255, 255,   0,   0,
    

### trainingdata_add.pickle로 저장

In [14]:
import pickle
with open('trainingdata_add2.pickle','wb') as fw:
    pickle.dump(data_additional, fw)

### 기존 데이터와 합쳐서 저장

In [15]:
with open('trainingdata.pickle', 'rb') as fr:
    data = pickle.load(fr)

In [16]:
len(data)

6220

In [69]:
len(data_additional)

1124

In [71]:
data_all = data + data_additional
len(data_all)

7344

In [72]:
with open('trainingdata_all.pickle','wb') as fw:
    pickle.dump(data_all, fw)